## Prepare - Station Data.

As the data is stored in a number of different .csv files, we do need to prepare the data before we can start process/clean it.
In this python worksheet, we aim to achieve the following:
1. Standarize the column names for all the sheets. 
2. Combine all the station data into one .csv file.

In [1]:
# Installing required libraries and setting the panda's display options.
import numpy as np
import pandas as pd
import os

pd.set_option('display.max_colwidth', 999)
pd.set_option('display.max_rows', 9999)

In [2]:
# We will begin by creating a pandas dataframe to store the file name, column names in a list and record numbers.
# This is just to look at the column names across files.

path = r"C:\Users\utsav\OneDrive\Documents\ML\Google Data Analytics Course\Course 8 - Capstone\Case Study 1\data\Unzipped\Combined\stations"
os.chdir(path)
df = pd.DataFrame(columns = ['file_name', 'col_names', 'num_records'])


In [3]:
files = 0

for file in os.listdir():
    file_path = f"{path}\{file}"
    with open(file_path, 'r') as f:
        print(f'File Number Opening: {files+1}')
        cols = f.readline()
        cols = cols[:-2]
        num_records = sum(1 for line in f) - 1
        cols_list = cols.split(',')
        df.at[files,'file_name'] = file
        df.at[files,'col_names'] = cols_list
        df.at[files,'num_records'] = num_records
        files += 1


File Number Opening: 1
File Number Opening: 2
File Number Opening: 3
File Number Opening: 4
File Number Opening: 5
File Number Opening: 6
File Number Opening: 7
File Number Opening: 8
File Number Opening: 9


In [4]:
sum(df['num_records'].astype('int'))

4229

In [5]:
print(f"Total number of records: {sum(df['num_records'].astype('int'))}")
df

Total number of records: 4229


,file_name,col_names,num_records
0,Divvy_Stations_2013.csv,"[id, name, latitude, longitude, dpcapacity, landmark, online_dat]",299
1,Divvy_Stations_2014-Q1Q2.csv,"[id, name, latitude, longitude, dpcapacity, online_dat]",299
2,Divvy_Stations_2014-Q3Q4.csv,"[id, name, latitude, longitude, dpcapacity, online_dat]",299
3,Divvy_Stations_2015.csv,"[id, name, latitude, longitude, dpcapacity, landmar]",473
4,Divvy_Stations_2016_Q1Q2.csv,"[id, name, latitude, longitude, dpcapacity, online_dat]",534
5,Divvy_Stations_2016_Q3.csv,"[id, name, latitude, longitude, dpcapacity, online_dat]",580
6,Divvy_Stations_2016_Q4.csv,"[id, name, latitude, longitude, dpcapacity, online_dat]",580
7,Divvy_Stations_2017_Q1Q2.csv,"[id, name, city, latitude, longitude, dpcapacity, online_dat]",581
8,Divvy_Stations_2017_Q3Q4.csv,"[id, name, city, latitude, longitude, dpcapacity, online_dat]",584


In [6]:
'''
This dataframe shows us the column names used in different files. To combine the data into a single .csv file, we need to make
sure that the column names are consistant. Being, only 9 files, we can do so by manually opening each file in spreadsheet
program and modifying the column names. 

After complete above step, we are ready to combine the data into one. We will include the file name as well, just to keep
track to where the observation coming from. 
'''
stations_list = []
for f in os.listdir():
    station_df = pd.read_csv(f, sep=',')
    station_df['file'] = f.split('/')[-1]
    stations_list.append(station_df)
    
stations_combined_data = pd.concat(stations_list, ignore_index=True, sort=True)

In [7]:
stations_combined_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4238 entries, 0 to 4237
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   city         1167 non-null   object 
 1   dpcapacity   4238 non-null   int64  
 2   file         4238 non-null   object 
 3   id           4238 non-null   int64  
 4   landmark     774 non-null    float64
 5   latitude     4238 non-null   float64
 6   longitude    4238 non-null   float64
 7   name         4238 non-null   object 
 8   online_date  3764 non-null   object 
dtypes: float64(3), int64(2), object(4)
memory usage: 298.1+ KB


In [8]:
# Making the columns consistent.
decimals = pd.Series([5, 5], index=['latitude', 'longitude'])
stations_combined_data = stations_combined_data.round(decimals)
stations_combined_data['online_date'] = stations_combined_data['online_date'].str[:9]

In [9]:
stations_combined_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4238 entries, 0 to 4237
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   city         1167 non-null   object 
 1   dpcapacity   4238 non-null   int64  
 2   file         4238 non-null   object 
 3   id           4238 non-null   int64  
 4   landmark     774 non-null    float64
 5   latitude     4238 non-null   float64
 6   longitude    4238 non-null   float64
 7   name         4238 non-null   object 
 8   online_date  3764 non-null   object 
dtypes: float64(3), int64(2), object(4)
memory usage: 298.1+ KB


In [10]:
# Reorganizing data and removing the duplicates.
stations_combined_data.drop_duplicates(subset = ['dpcapacity', 'id', 'latitude', 'longitude', 'name'], keep = 'last', inplace = True)
cols = ['id','name', 'latitude','longitude','dpcapacity','online_date','file']
stations_combined_data = stations_combined_data[cols]


In [11]:
# Sorting by station id.
stations_combined_data.sort_values(by=['id','name'], inplace = True, ignore_index = True)

In [12]:
stations_combined_data

,id,name,latitude,longitude,dpcapacity,online_date,file
0,2,Buckingham Fountain,41.87267,-87.62397,35,5/08/2015,Divvy_Stations_2016_Q1Q2.csv
1,2,Buckingham Fountain,41.88106,-87.61949,27,6/10/2013,Divvy_Stations_2017_Q1Q2.csv
2,2,Buckingham Fountain,41.87639,-87.62033,27,6/10/2013,Divvy_Stations_2017_Q3Q4.csv
3,2,Michigan Ave & Balbo Ave,41.87229,-87.62409,35,NaN,Divvy_Stations_2015.csv
4,2,Michigan Ave & Balbo Ave,41.87264,-87.62398,35,5/08/2015,Divvy_Stations_2016_Q4.csv
5,3,Shedd Aquarium,41.86723,-87.61536,31,6/10/2013,Divvy_Stations_2017_Q1Q2.csv
6,3,Shedd Aquarium,41.86723,-87.61536,55,6/10/2013,Divvy_Stations_2017_Q3Q4.csv
7,4,Burnham Harbor,41.85627,-87.61335,23,6/10/2013,Divvy_Stations_2017_Q3Q4.csv
8,5,State St & Harrison St,41.87396,-87.62774,19,6/10/2013,Divvy_Stations_2014-Q3Q4.csv
9,5,State St & Harrison St,41.87405,-87.62772,23,6/10/2013,Divvy_Stations_2017_Q3Q4.csv


In [13]:
stations_combined_data.to_csv('combined_station_data.csv')

### Remarks:
There seems to be a few duplicates in the table. This is because there are a few minor differences in other fields like dcapacity, latitudes or longitudes and even the names of a few stations have been changed.
We will clean and take care of these issues in the process part of the project.